In [ ]:
import pandas as pd
import numpy as np
import regex as re
from pandarallel import pandarallel
# import concurrent.futures
# from bs4 import BeautifulSoup
# import requests
# from urllib.parse import urlparse
# import os
# from pathlib import Path
# from natsort import natsorted, ns
import glob
import pydash as dash
# import rapidfuzz as fuzz
# from rapidfuzz.process import extractOne
# from rapidfuzz.string_metric import levenshtein, normalized_levenshtein
# from rapidfuzz.fuzz import ratio
# from rapidfuzz import utils as fuzz_utils
import itertools as it
# import tfidf_matcher as matcher
from ftfy import fix_text
# from textblob import TextBlob, Word, Blobber
# from collections import namedtuple, defaultdict, deque

pandarallel.initialize(progress_bar=False, verbose=1)
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.float_format', lambda x: '%.0f' % x)

from mlxtend.frequent_patterns import apriori, association_rules

In [112]:
df_in = pd.read_excel('infiles/odoo_orders_all.xlsx', sheet_name='update')
# df_in = pd.read_csv('/Users/ash/Downloads/orders_export.csv')

In [125]:
df = df_in.copy()

In [126]:
df.shape

(79826, 4)

In [127]:
# def analyze(*args, **kwargs):
#     """
#     analyze creates basket analysis with support and lift metrics
#     Arguments:
#     [
#         args[0] = DataFrame,
#         args[1] = Invoice or Order Id,
#         args[2] = Barcode, or Comparible Basket Item
#         args[3] = Quantity sold of each item within each order
#     ]
#     """

#     def encodeUnits(x):
#         if x <= 0:
#             return 0
#         if x >= 1:
#             return 1

#     def format_data(*args):
#         if type(args[1]) == list:
#             df = (args[0].groupby([args[1][1], args[1][2]])[args[1][3]].sum().unstack().reset_index().fillna(0).set_index(args[1][1]))
#             print('yes')
#         else:
#             print(print(args[1][1]))
#             df = (args[0].groupby([args[1][1], args[1][2]])[args[1][3]].sum().unstack().reset_index().fillna(0).set_index(args[1][1]))

#         df_enc = df.parallel_applymap(encodeUnits)
#         df_enc_plus = df_enc[(df_enc > 0).sum(axis=1) >= 2]
#         return df_enc_plus
#     def apply_apriori(df, **kwargs):
#         bk = apriori(df, min_support=kwargs.get('min_support', 0.002), use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

#         assoc = association_rules(bk, metric='lift', min_threshold=1).sort_values(by='lift', ascending=False).reset_index(drop=True)
#         assoc[['antecedents', 'consequents']] = assoc[['antecedents', 'consequents']].applymap(lambda x: ', '.join(x))

#         return assoc

#     basket = format_data(*args)
#     bk_df = apply_apriori(basket)
#     return bk_df

# def formatDF(*args, **kwargs):
#     col_names = ['Name', 'Lineitem sku', 'Lineitem name', 'Lineitem quantity']
#     df = args[0]
#     df_out = df[col_names].copy()

#     df_out['Name'] = df_out['Name'].astype(str)
#     df_out['Lineitem quantity'] = df_out['Lineitem quantity'].astype(np.int32)
#     return df_out

In [132]:
def analyze(*args, **kwargs):
    """
    analyze creates basket analysis with support and lift metrics
    Arguments:
    [
        args[0] = DataFrame,
        args[1] = Invoice or Order Id,
        args[2] = Barcode, or Comparible Basket Item
        args[3] = Quantity sold of each item within each order
    ]
    """

    def encodeUnits(x):
        if x <= 0:
            return False
        if x >= 1:
            return True

    def format_data(*args):
        cols = args[0].columns.to_list()
        df = args[0].copy()
        df[cols[3]] = df[cols[3]].astype(float)
        df = (args[0].groupby([cols[0], cols[1]])[cols[3]].sum().unstack().reset_index().fillna(0).set_index(cols[0]))
        df_enc = df.parallel_applymap(encodeUnits)
        df_enc_plus = df_enc[(df_enc > 0).sum(axis=1) >= 2]
        return df_enc_plus
    
    def apply_apriori(df, **kwargs):
        bk = apriori(df, min_support=kwargs.get('min_support', 0.002), use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

        assoc = association_rules(bk, metric='lift', min_threshold=1).sort_values(by='lift', ascending=False).reset_index(drop=True)
        assoc[['antecedents', 'consequents']] = assoc[['antecedents', 'consequents']].applymap(lambda x: ', '.join(x))

        return assoc

    basket = format_data(*args)
    bk_df = apply_apriori(basket)
    return bk_df

In [133]:
bk = analyze(df)

In [118]:

def formatDF(*args, **kwargs):
    col_names = ['Name', 'Lineitem sku', 'Lineitem name', 'Lineitem quantity']
    df = args[0]
    df_out = df[col_names].copy()

    df_out['Name'] = df_out['Name'].astype(str)
    df_out['Lineitem quantity'] = df_out['Lineitem quantity'].astype(np.int32)
    return df_out

In [123]:
def encodeUnits(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

def format_data(*args):
    # cols = ['orderId', 'stockCode', 'items', 'itemQty']
    cols = args[0].columns.to_list()
    df_pre = args[0].copy()
    df_pre[cols[3]] = df_pre[cols[3]].astype(float)
    df = (df_pre.groupby([cols[0], cols[1]])[cols[3]].sum().unstack().reset_index().fillna(0).set_index(cols[0]))
    # df = (args[0].groupby([cols[0], cols[1]])[cols[2]].sum().unstack().reset_index().fillna(0).set_index(cols[1]))
    
    df_enc = df.parallel_applymap(encodeUnits)
    df_enc_plus = df_enc[(df_enc > 0).sum(axis=1) >= 2]
    return df_enc_plus

In [120]:
cols = df.columns.to_list()
df['itemQty'] = df['itemQty'].astype(float)

In [121]:
dfg = df.groupby([cols[0], cols[1]])[cols[3]].sum().unstack().reset_index().fillna(0).set_index(cols[0])

In [122]:
df_gr = df.groupby([cols[0], cols[1]])[[cols[3]]].parallel_apply(lambda x: np.sum(x)).unstack().reset_index().fillna(0).set_index(cols[0])

In [92]:
# main = analyze(df, df.columns, min_support=0.002)
main = format_data(df)

KeyError: "None of ['stockCode'] are in the columns"

In [83]:
df.columns
argys = ['orderId', 'stockCode', 'items', 'itemQty']

In [ ]:
main

In [ ]:
df['itemQty'] = df['itemQty'].astype(np.int32)
df['orderId'] = df['orderId'].astype(str)

In [ ]:
items = df.pop('items')

In [ ]:
df.columns

In [ ]:
def analyze(*args, **kwargs):
    """
    analyze creates basket analysis with support and lift metrics
    Arguments:
    [
        args[0] = DataFrame,
        args[1] = Invoice or Order Id,
        args[2] = Barcode, or Comparible Basket Item
        args[3] = Quantity sold of each item within each order
    ]
    """

    def encodeUnits(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    def format_data(*args):
        df = (args[0].groupby([args[1], args[2]])[args[3]].sum().unstack().reset_index().fillna(0).set_index(args[1]))
        df_enc = df.parallel_applymap(encodeUnits)
        df_enc_plus = df_enc[(df_enc > 0).sum(axis=1) >= 2]
        return df_enc_plus

    def apply_apriori(df, **kwargs):
        bk = apriori(df, min_support=kwargs.get('min_support', .002), use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
        assoc = association_rules(bk, metric='lift', min_threshold=1).sort_values(by='lift', ascending=False).reset_index(drop=True)
        assoc[['antecedents', 'consequents']] = assoc[['antecedents', 'consequents']].applymap(lambda x: ', '.join(x))
        return assoc

    basket = format_data(*args)
    bk_df = apply_apriori(basket)
    return bk_df

In [ ]:
basket = analyze(df, 'orderId', 'stockCode', 'itemQty', min_support=0.002)

In [ ]:
def encodeUnits(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1


bkt = (df.groupby(['orderId', 'stockCode'])['itemQty'].sum().unstack().reset_index().fillna(0).set_index('orderId'))
bkt.info()
bkt_enc = bkt.parallel_applymap(encodeUnits)
bkt_enc.head()
bkt_enc_plus = bkt_enc[(bkt_enc > 0).sum(axis=1) >= 2]
# from apyori import apriori as apy
# freq_itms = apriori(bkt_enc_plus, min_support=.0030, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
freq_itms_two = apriori(bkt_enc_plus, min_support=.0020, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
# rules= apy(bkt_enc_plus, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
# assoc_rules = list(rules)
# assoc_rules

# freq_itms['length'] = freq_itms['itemsets'].apply(lambda x: len(x))
freq_itms_two['length'] = freq_itms_two['itemsets'].apply(lambda x: len(x))

assoc = association_rules(freq_itms_two, metric='lift', min_threshold=1).sort_values(by='lift', ascending=False).reset_index(drop=True)
assoc[['antecedents', 'consequents']] = assoc[['antecedents', 'consequents']].applymap(lambda x: ', '.join(x))
assoc.shape
from natsort import natsorted
assoc.insert(2, 'pairs', np.nan)
# assoc['dupes'] = assoc.loc[assoc[['antecedents', 'consequents']].apply(lambda x: natsorted(list(x)), axis=1).duplicated(keep=False)] #.sort_values(by=['antecedents', 'consequents'], ascending=[True, False]) #.duplicated(subset=['antecedents', 'consequents'])
assoc['pairs'] = assoc[['antecedents', 'consequents']].apply(lambda x: natsorted(list(x)), axis=1)
assoc['pairs'] = assoc['pairs'].str.join(', ')

In [ ]:
def analyze(*args, **kwargs):
    """
    analyze creates a basket analysis
    args[str] = [
        args[00] = Invoice # or Order Id,
        args[01] = Sku, Barcode, or Comparible Data
    ]
    """
    df = (args[0].groupby(['orderId', 'stockCode'])['itemQty'].sum().unstack().reset_index().fillna(0).set_index('orderId'))

In [ ]:
af = assoc.sort_values(by=['lift', 'confidence'], ascending=[False, False]).drop_duplicates(subset='pairs', ignore_index=True).copy()

In [ ]:
af

In [ ]:
with pd.ExcelWriter('infiles/odoo_order_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
    af.to_excel(writer, index = False, sheet_name='results_002_noDupes')

In [ ]:
# assoc[['antecedents', 'consequents']].apply(list, axis=1)